#Imports

In [0]:
#https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

%tensorflow_version 2.x
import tensorflow as tf
from array import *
!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tf-nightly
import tensorflow_datasets as tfds
import tensorflow as t

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TensorFlow 2.x selected.
     |████████████████████████████████| 3.1MB 4.8MB/s 
     |████████████████████████████████| 464.2MB 30kB/s 
     |████████████████████████████████| 460kB 42.4MB/s 
     |████████████████████████████████| 3.9MB 37.8MB/s 
ERROR: tensorflow-federated 0.11.0 requires enum34~=1.1, which is not installed.
ERROR: tensorflow 2.1.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement attrs~=18.2, but you'll have attrs 19.3.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement cachetools~=3.1.1, but you'll have cachetools 4.0.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement grpcio~=1.24.3, but you'll have grpcio 1.26.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement portpicker~=1.3.1, but you'll have portpicker 1.2.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement tensorflow~=2.0.0, but you'll

Version:  2.1.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [0]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

dataset, info = tfds.load(name="imdb_reviews", with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch



<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),  
    tf.keras.layers.Dense(1, activation='sigmoid'),
])




model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    validation_steps=10
                   )   
   

results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))  

temp_list = [] 
temp_list.append("The best movie of the history")


predictions = model.predict(temp_list)
predicted_labels = ""
CLASS_NAMES = ['positive','negative']

for i in (-predictions[0]).argsort()[:2]:
  predicted_labels += CLASS_NAMES[i] + " (" + str(np.around(predictions[0][i], decimals=2)) + ")\n"

print('\nLa prediccion es:\n', predicted_labels)

Epoch 1/20
30/30 [==============================] - 5s 169ms/step - loss: 0.1362 - accuracy: 0.9569 - val_loss: 0.3163 - val_accuracy: 0.8727
Epoch 2/20
30/30 [==============================] - 5s 164ms/step - loss: 0.1265 - accuracy: 0.9601 - val_loss: 0.3131 - val_accuracy: 0.8742
Epoch 3/20
30/30 [==============================] - 5s 161ms/step - loss: 0.1193 - accuracy: 0.9634 - val_loss: 0.3190 - val_accuracy: 0.8738
Epoch 4/20
30/30 [==============================] - 5s 169ms/step - loss: 0.1105 - accuracy: 0.9672 - val_loss: 0.3196 - val_accuracy: 0.8717
Epoch 5/20
30/30 [==============================] - 5s 166ms/step - loss: 0.1055 - accuracy: 0.9695 - val_loss: 0.3255 - val_accuracy: 0.8742
Epoch 6/20
30/30 [==============================] - 5s 173ms/step - loss: 0.0980 - accuracy: 0.9719 - val_loss: 0.3285 - val_accuracy: 0.8693
Epoch 7/20
30/30 [==============================] - 5s 164ms/step - loss: 0.0921 - accuracy: 0.9737 - val_loss: 0.3342 - val_accuracy: 0.8703
Epoch 